In [1]:
%%capture
%pip install -U transformers accelerate

## vison model form input in kaggel laden 

In [2]:

# import torch
# from transformers import MllamaForConditionalGeneration, AutoProcessor

# base_model = "/kaggle/input/llama-3.2-vision/transformers/11b-vision-instruct/1"

# processor = AutoProcessor.from_pretrained(base_model)

# model = MllamaForConditionalGeneration.from_pretrained(
#     base_model,
#     low_cpu_mem_usage=True,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

## testin basic unfintuedned model with thes mp4 to see performace bevore finetuneing

In [ ]:
import torch
import requests
from PIL import Image
import cv2

from transformers import MllamaForConditionalGeneration, AutoProcessor

# === Modellpfad & Laden ===
base_model = "/kaggle/input/llama-3.2-vision/transformers/11b-vision-instruct/1"
processor = AutoProcessor.from_pretrained(base_model)
model = MllamaForConditionalGeneration.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Clear CUDA cache before starting
torch.cuda.empty_cache()

# === 1. Lade lokalen Video-Frame ===
def extract_frame_from_video(video_path, frame_number=0):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        raise ValueError("Kein Frame gefunden.")
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return Image.fromarray(image)

video_path = "/kaggle/input/testingthemodel/test_file_DGL.mp4"  # ← Pfad zu deiner Datei!
image = extract_frame_from_video(video_path, frame_number=10)  # z.B. Frame 10

# Optional verkleinern, um Speicher zu sparen
image = image.resize((224, 224))

# === 2. Prompt bauen ===
messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "Analysiere die Bildsequenz. Sie besteht aus mehreren Einzelbildern, die aus einem Video extrahiert wurden.  Die Bilder zeigen die zeitliche Entwicklung einer Gebärde, angeordnet von links nach rechts. Gib den gesprochenen Inhalt in Hochdeutsch zurück."}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)

# === 3. Verarbeitung vorbereiten ===
inputs = processor(image, input_text, return_tensors="pt").to(model.device)
inputs = {k: v.to(dtype=torch.float16) if v.dtype == torch.float32 else v for k, v in inputs.items()}

# === 4. Modell starten ===
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=60)

# === 5. Ausgabe anzeigen ===
print(processor.decode(output[0], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

user

Was sagt die Person in Gebärdensprache? Antworte in Hochdeutsch.assistant

Die Person in Gebärdensprache sagt: "Ich liebe dich."
